In [22]:
import numpy as np
import control.matlab as m
import matplotlib.pyplot as plt

from pathlib import Path

PATH_IMAGES = Path('images')


# Dados de Entrada

In [23]:
fpass   = 2772    #[Hz] 
fstop   = 2272    #[Hz]
apass   = -3      #[dB]
astop   = -18     #[dB]
omegarH = fpass/fstop
omega0  = 2*np.pi*fpass

# Funções úteis

In [39]:
def epsf(a:float)->float:
    return np.sqrt(10**(-0.1*a) -1)

'''
FILTRO DE BUTTERWORTH
'''
def thetam (n:int)->np.ndarray:
    aux     = 0 if n%2 == 0 else 1
    mfinal  = (n-aux)/2 - 1
    m       = np.arange(0, mfinal+1)
    
    return np.pi*(2*m + n + 1)/(2*n)

def FTb(n:int, R:float, w0:float = 1, PB:bool = True, BW:float = None)-> m.TransferFunction:
        print(5*'-'+'BUTTERWORTH FILTER'+5*'-')
        if n > 4:
            print(f'--> Filtro reduzido de {n}th para 4th ordem')
            n = 4
        sigma_m = R*np.cos(thetam(n))
        omega_m = R*np.sin(thetam(n))
        exp     = 0 if n%2 == 0 else 1
        


        s  = m.tf('s')
        if BW == None:
            S = (s/w0)**(1 if PB else -1)
        else:
            S = ( (s**2 + w0**2)/BW )**(1 if PB else -1)
        
        B1 = -2*sigma_m
        B2 = sigma_m**2 + omega_m**2

        H = R**(exp)*np.prod(B2)/((S+R)**exp)
        for i in range(len(sigma_m)):
            H*=1/(S**2 + B1[i]*S + B2[i])

        # Prints
        print(f'theta_m\t= {np.round(thetam(n),3)} [rad]')
        print(f'sigma_m\t= {np.round(sigma_m,3)}')
        print(f'omega_m\t= {np.round(omega_m,3)} [rad/s]') 
        if not n%2 == 0:
            print(f'sigma_r\t= {-R}')
        print(f'Hb\t= {H}')
        return H, (sigma_m, omega_m)
        



# 1. Filtro de Butterworth

### 1.1 Filtro passa-baixa normalizado

In [41]:
# fator de ajuste
eps = epsf(apass)

# ordem do filtro
nb  = np.log10(epsf(astop)/epsf(apass))/(2*np.log10(omegarH))
n   =  round(nb)+1 if not nb%1 == 0 else nb

# alocação dos polos
R = eps**(-1/n)

print(f'epsilon\t= {round(eps,3)}')
print(f'nb\t= {round(nb,3)} ({n}th order)\n')

# função de transferência
H,_ = FTb(n,R)


epsilon	= 0.998
nb	= 5.195 (6th order)

-----BUTTERWORTH FILTER-----
--> Filtro reduzido de 6th para 4th ordem
theta_m	= [1.963 2.749] [rad]
sigma_m	= [-0.383 -0.924]
omega_m	= [0.924 0.383] [rad/s]
Hb	= 
                    1.002
---------------------------------------------
s^4 + 2.614 s^3 + 3.417 s^2 + 2.616 s + 1.002



In [26]:
import control.matlab as m
s = m.tf('s')

S = s/2
H = 15/(S + 2)
print(6%2)

0
